In [1]:
import pickle
import pandas as pd
import re
import datetime
import warnings

In [5]:
with open("./data/rocketpunch_full.pkl","rb") as f:
    rocket_punch=pickle.load(f)

In [40]:
rocket_punch.columns 
#description 이름이 같은 중복 컬럼 2개 존재

Index(['company_id', 'company_name', 'description', 'job_id', 'job_title',
       'job_info', 'job_date', '@context', '@type', 'datePosted',
       'description', 'hiringOrganization', 'title', 'url', 'jobLocation',
       'employmentType', 'experienceRequirements', 'occupationalCategory',
       'requirement', 'specialties', 'validThrough', 'baseSalary',
       'jobLocationType'],
      dtype='object')

In [6]:
# 중복 컬럼 이름 변경
columns = rocket_punch.columns.tolist()
last_index = len(columns) - 1 - columns[::-1].index('description')
columns[last_index] = 'description2'  

# 변경된 칼럼 리스트를 다시 DataFrame에 할당
rocket_punch.columns = columns

In [15]:
# 필요 컬럼만 추출
filter_rocket_punch = rocket_punch[['company_name', 'job_title', 'experienceRequirements', 'job_date', 'specialties', 'description2', 'description' , 'requirement']]

In [16]:
#null 값 찾기
for i in filter_rocket_punch.iterrows():
    if i is None:
        print(i)
    else:
        pass


In [17]:
# 컬럼 이름 변경
filter_rocket_punch.columns = ["company_name", \
"title", \
"career_raw", \
"date_raw", \
"tech_stack", \
"description", \
"company_description", 
"required"    
    ]

In [18]:
filter_rocket_punch.columns # 총 컬럼 수 : 9

Index(['company_name', 'title', 'career_raw', 'date_raw', 'tech_stack',
       'description', 'company_description', 'required'],
      dtype='object')

In [ ]:
#filter_rocket_punch.head(20)

In [19]:
# 각 행을 확인하고 조건에 따라 'date' 열 설정
for idx, dr in enumerate(filter_rocket_punch['date_raw']):
    # 수시채용은 9999-12-30
    if "수시채용" in dr:
        filter_rocket_punch.loc[idx, 'date'] = '9999-12-30'
    # 일시채용 : mm/dd 마감 > 2024-mm-dd로 변경
    else:
        mmdd = dr.split()[0].split('/')[0:2]
        filter_rocket_punch.loc[idx, 'date'] = f"2024-{mmdd[0]}-{mmdd[1]}"


/tmp/ipykernel_1650/3678245174.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_rocket_punch.loc[idx, 'date'] = '9999-12-30'


In [ ]:
#display(filter_rocket_punch['date'])

In [49]:
#filter_rocket_punch['career_raw'].value_counts() 

career_raw
[경력]        198
[신입, 경력]     72
[신입]          8
Name: count, dtype: int64

In [20]:
# 경력 1, 신입0. 칼럼에 두 값이 모두 있을 경우, 신입
filter_rocket_punch['career'] = filter_rocket_punch['career_raw'].apply(lambda x : 0 if '신입' in x else 1)

/tmp/ipykernel_1650/681031454.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_rocket_punch['career'] = filter_rocket_punch['career_raw'].apply(lambda x : 0 if '신입' in x else 1)


In [21]:
# job_site_code 추가
filter_rocket_punch['job_site_code'] = 4

/tmp/ipykernel_1650/122210433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_rocket_punch['job_site_code'] = 4


In [22]:
filter_rocket_punch.head()

,company_name,title,career_raw,date_raw,tech_stack,description,company_description,required,date,career,job_site_code
0,(주)휴가중,[온베케이션] 프론트엔드 웹 개발자 (React),[경력],수시채용,"[프론트엔드 개발, Git, Amazon Web Services]",온베케이션은 스타트업의 기발함과 여행업의 설렘으로 새로운 가치를 만들어 나갑니다.<...,Travel Tech 를 영역에서 다양한 혁신 시도를 통해 더 많은 여행자들의 감동...,[ 주요 업무 ]• React/Next.js 기반의 백오피스 개발• 서비스 운영 이...,9999-12-30,1,4
1,(주)휴가중,[온베케이션] Flutter Mobile App Developer,"[신입, 경력]",수시채용,"[GitHub, android, Django, iOS, Java, Kotlin, P...",온베케이션은 스타트업의 기발함과 여행업의 설렘으로 새로운 가치를 만들어 나갑니다.<...,Travel Tech 를 영역에서 다양한 혁신 시도를 통해 더 많은 여행자들의 감동...,[ 주요 업무 ]• Flutter 기반의 신규 앱 개발 및 운영• 빌드/배포/테스트...,9999-12-30,0,4
2,(주)휴가중,[온베케이션] React Native Mobile App Developer,"[신입, 경력]",수시채용,"[Git, android, iOS, Java, Python, SQL, TypeScr...",온베케이션은 스타트업의 기발함과 여행업의 설렘으로 새로운 가치를 만들어 나갑니다.<...,Travel Tech 를 영역에서 다양한 혁신 시도를 통해 더 많은 여행자들의 감동...,[ 주요 업무 ]• React Native 기반의 신규 앱 개발 및 운영• 빌드/배...,9999-12-30,0,4
3,아우라,웹 프론트 엔지니어,[경력],05/01 마감,"[프론트엔드 개발, Git, GitLab, Flutter & Dart, HTML, ...",AI 기반 블록체인 민팅 프로젝트 프론트 개발<br />해외 시장을 대상으로 한 블...,피규어 및 굿즈 제작,"[기본사항]- 새로운 기술 트렌드에 적극적:Flutter, React, Vue.js...",2024-05-01,1,4
4,슬로우 플랫폼,모바일 SW 개발자,"[신입, 경력]",수시채용,"[안드로이드, Java, Kotlin, Flutter & Dart, MVVM, Cl...",- 스마트에이징의 새로운 길을 여는 슬로우 서비스를 앞에서 이끌어갈 모바일 애플리케...,시니어를 위한 스마트 에이징 플랫폼,슬로우플랫폼은 'SlowTalk'이라는 세대공감 플랫폼 서비스를 준비 중입니다. 이...,9999-12-30,0,4


In [23]:
# 불필요한 열 제거
filter_rocket_punch = filter_rocket_punch.drop(["career_raw", "date_raw"], axis=1)

In [ ]:
#filter_rocket_punch.head()

In [28]:
# 특수문자 제거
for col in ['required', 'company_description', 'description', 'title']: 
    filter_rocket_punch[col]=filter_rocket_punch[col].apply(lambda x : x.encode('cp949', 'ignore').decode('cp949') )

In [29]:
# pickle 저장
with open("./data/filter_rocketpunch.pkl", "wb") as f:
    pickle.dump(filter_rocket_punch, f)

data to DB insert

In [3]:
import pymysql
import pickle
import os
from dotenv import load_dotenv

In [30]:
# pickle 가져오기
with open("./data/filter_rocketpunch.pkl", "rb") as f:
    filter_rocketpunch = pickle.load(f)

In [31]:
filter_rocketpunch.columns

Index(['company_name', 'title', 'tech_stack', 'description',
       'company_description', 'required', 'date', 'career', 'job_site_code'],
      dtype='object')

In [57]:
# db에 넣기 위해 순서 변경
new_order = [
    'job_site_code', 'company_name', 'title', 'career', 'date', 'tech_stack', 
    'description', 'company_description', 'required'
]

In [58]:
filter_rocketpunch=filter_rocketpunch[new_order]

In [ ]:
#filter_rocketpunch.head(5)

In [5]:
db_info = {}
with open("./db_info.env", "r") as f:
  for line in f:
    key, value = line.strip().split("=")
    db_info[key] = value

In [11]:

conn = pymysql.connect(host=db_info['DB_HOST'], user=db_info['DB_USER'], password=db_info['DB_PASS'], db=db_info['DB_NAME'], port=int(db_info['DB_PORT']), charset='utf8')
sql="""
INSERT INTO job_data (job_site_code, company_name, title, career, date_until, tech_stack, description, company_description, required)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
cursor = conn.cursor(pymysql.cursors.DictCursor)

In [ ]:
for idx, row in filter_rocketpunch.iterrows():
    try:
        if len(row['tech_stack']) < 300:
            data_tuple = (row['job_site_code'], row['company_name'], row['title'], row['career'],
                          row['date'], row['tech_stack'], row['description'], 
                          row['company_description'], row['required'])  # Assuming 'company_description' was intended here
            cursor.execute(sql, data_tuple)
    except Exception as e:  
        print(f"Error: {e}")  
        if len(row['tech_stack']) > 300: # varchar(300)이 넘는 이슈, 제외
            #print(idx) 
            pass

In [73]:
#filter_rocketpunch.iloc[1]['tech_stack']

'GitHub, android, Django, iOS, Java, Kotlin, Python, SQL, TypeScript, AWS, 웹 프론트엔드 개발(HTML5/CSS3/JavaScript/Vue.js), 프론트엔드 개발'